In [ ]:
## !pip install psycopg2

In [1]:
import psycopg2
import pandas as pd

### Create connection to database

In [5]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=hockey_test user=postgres password=password")
except psycopg2.Error as e:
    print("Error: Could not connect to database")
    print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the database")
    print(e)

conn.set_session(autocommit=True)

#### create table for dim_dates

In [6]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS dim_dates (\
                id int,\
                date date,\
                day varchar,\
                week int,\
                is_weekend int,\
                is_holiday int);")
    
except psycopg2.Error as e:
    print("Error: Could not create table")
    print(e)

#### Insert values into table dim_dates

In [7]:
## load DF for dates
df_dates = pd.read_csv("/Users/mattrende/Documents/Coding/hockey-reference/YouTube Project/csv/out/dim_dates.csv", parse_dates=["Date"])
df_dates.head(2)

,ID,Date,Day,Week,Is_Weekend,Is_Holiday
0,20190101,2019-01-01,Tuesday,1,0,1
1,20190102,2019-01-02,Wednesday,1,0,0


In [8]:
date_insert = ("""INSERT INTO dim_dates(
               id,
               date,
               day,
               week,
               is_weekend,
               is_holiday)
               VALUES (%s, %s, %s, %s, %s, %s)
               """)

In [9]:
try:
    for i, row in df_dates.iterrows():
        #print(list(row))
        cur.execute(date_insert, list(row))

except psycopg2.Error as e:
    print("Error: could not insert rows")
    print(e)

In [10]:
try: 
    cur.execute("SELECT * From dim_dates;")

except psycopg2.Error as e:
    print("Select Error")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(20190101, datetime.date(2019, 1, 1), 'Tuesday', 1, 0, 1)
(20190102, datetime.date(2019, 1, 2), 'Wednesday', 1, 0, 0)
(20190103, datetime.date(2019, 1, 3), 'Thursday', 1, 0, 0)
(20190104, datetime.date(2019, 1, 4), 'Friday', 1, 0, 0)
(20190105, datetime.date(2019, 1, 5), 'Saturday', 1, 1, 0)
(20190106, datetime.date(2019, 1, 6), 'Sunday', 1, 1, 0)
(20190107, datetime.date(2019, 1, 7), 'Monday', 2, 0, 0)
(20190108, datetime.date(2019, 1, 8), 'Tuesday', 2, 0, 0)
(20190109, datetime.date(2019, 1, 9), 'Wednesday', 2, 0, 0)
(20190110, datetime.date(2019, 1, 10), 'Thursday', 2, 0, 0)
(20190111, datetime.date(2019, 1, 11), 'Friday', 2, 0, 0)
(20190112, datetime.date(2019, 1, 12), 'Saturday', 2, 1, 0)
(20190113, datetime.date(2019, 1, 13), 'Sunday', 2, 1, 0)
(20190114, datetime.date(2019, 1, 14), 'Monday', 3, 0, 0)
(20190115, datetime.date(2019, 1, 15), 'Tuesday', 3, 0, 0)
(20190116, datetime.date(2019, 1, 16), 'Wednesday', 3, 0, 0)
(20190117, datetime.date(2019, 1, 17), 'Thursday', 3, 0, 0)
(

### Close cursor and connection

In [11]:
cur.close()
conn.close()